### This notebook implements the equations in the Appendix "Adjusting Quadratic Mechanisms for Pre-Existing Cooperation" of the paper "Decentralized Society: Finding Web3’s Soul" by E. Glen Weyl, Puja Ohlhaver, and Vitalik Buterin. 

In [1]:
import numpy as np

In [2]:
# the agents
agents = ['A', 'B', 'S']
# the agents' respective contributions
contributions = [1, 2, 3]
# initialize a dictionary mapping each agent to their contribution
agents_to_contributions_dict = {}
# populate the dictionary
for agent_index, agent in enumerate(agents):
    agents_to_contributions_dict[agent] = contributions[agent_index]

# Simple Match

In [3]:
def simple_match(contributions):
    """
    :param contributions: An array of contributions
    :return: A simple match to the (presumed non-cooperating) contributions based on quadratic funding
    """
    
    return np.sum(np.sqrt(contributions))**2 - np.sum(contributions)

In [4]:
simple_match_output = simple_match(contributions=contributions)
print(f"The simple match is {simple_match_output}")

The simple match is 11.191508225450303


# Cluster Match

In [5]:
# initialize a dictionary mapping each cluster id to the agents in that cluster
clusters_to_agents_dict_single_membership = {}
# populate the dictionary
clusters_to_agents_dict_single_membership[0] = ['A', 'S']
clusters_to_agents_dict_single_membership[1] = ['B']

In [6]:
def get_contributions_from_cluster(agents_to_contributions_dict, clusters_to_agents_dict_single_membership, cluster_id):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict_single_membership: a dictionary mapping a cluster id to the agents in that cluster
    :param cluster_id: the id of the cluster for which we want a list of the contributions made by agents in cluster cluster_id
    :return: a list of the contributions made by agents in cluster cluster_id
    """

    # initialize the list of contributions from agents in the cluster
    contributions_from_cluster = [] 
    # get the agents in the cluster
    cluster_agents = clusters_to_agents_dict_single_membership[cluster_id] 
    for cluster_agent in cluster_agents:
        # get the contribution of the agent
        cluster_agent_contribution = agents_to_contributions_dict[cluster_agent]
        # add the contribution to the list
        contributions_from_cluster.append(cluster_agent_contribution)
    
    return contributions_from_cluster

In [7]:
def get_sum_of_all_contributions_from_agents_to_contributions_dict(agents_to_contributions_dict):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :return: the sum of all contributions from all agents
    """

    return sum(agents_to_contributions_dict.values())

In [8]:
def cluster_match(agents_to_contributions_dict, clusters_to_agents_dict_single_membership):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict_single_membership: a dictionary mapping a cluster id to the agents in that cluster
    :return: the cluster match
    """
    
    # the sum of the elements inside the parentheses of the Single Membership Cluster Match
    sum_sqrt_clusters_sums = 0
    for cluster_key, cluster_agents in clusters_to_agents_dict_single_membership.items():
        # the sum of the elements inside one of the square root terms in the parentheses
        current_cluster_sum = sum(get_contributions_from_cluster(agents_to_contributions_dict, clusters_to_agents_dict_single_membership, cluster_key))
        # the square root of the sum of the elements
        sqrt_current_cluster_sum = np.sqrt(current_cluster_sum)
        # add the square root of the sum of the elements to the total sum inside the parentheses
        sum_sqrt_clusters_sums += sqrt_current_cluster_sum
    # the sum in the parentheses squared
    sum_sqrt_clusters_sums_squared = np.power(sum_sqrt_clusters_sums, 2)
    # the sum of the contributions from the agents
    sum_contributions = get_sum_of_all_contributions_from_agents_to_contributions_dict(agents_to_contributions_dict)
    # the cluster match
    cluster_match = sum_sqrt_clusters_sums_squared - sum_contributions
    
    return cluster_match

In [9]:
cluster_match_output = cluster_match(agents_to_contributions_dict, clusters_to_agents_dict_single_membership)
print(f"The cluster match is {cluster_match_output}")

The cluster match is 5.65685424949238


# Offset Match

In [10]:
def offset_match(agents_to_contributions_dict, clusters_to_agents_dict_single_membership):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict_single_membership: a dictionary mapping a cluster id to the agents in that cluster
    :return: the offset match
    """
    
    # the sum of the elements inside the parentheses of the Single Membership Offset Match
    sum_sqrt_clusters_sums = 0    
    for cluster_key, cluster_agents in clusters_to_agents_dict_single_membership.items():
        # the contributions from agents in the current cluster
        contributions_from_cluster = get_contributions_from_cluster(agents_to_contributions_dict, clusters_to_agents_dict_single_membership, cluster_key)
        # the square root of each of the contributions from agents in the current cluster
        sqrt_current_cluster_agents = np.sqrt(contributions_from_cluster)
        # the sum of the square roots of each of the contributions from agents in the current cluster
        sum_sqrt_current_cluster_agents = np.sum(sqrt_current_cluster_agents)
        # the square root of the number of contributions/agents in the cluster
        sqrt_num_contributions_from_cluster = np.sqrt(len(contributions_from_cluster))
        # the current cluster term (one of the elements inside the parentheses)
        current_cluster_term = sum_sqrt_current_cluster_agents / sqrt_num_contributions_from_cluster
        # add the current cluster term to the sum of the elements inside the parentheses 
        sum_sqrt_clusters_sums += current_cluster_term
    # the sum in the parentheses squared
    sum_sqrt_clusters_sums_squared = np.power(sum_sqrt_clusters_sums, 2)
    # the sum of the contributions from the agents
    sum_contributions = get_sum_of_all_contributions_from_agents_to_contributions_dict(agents_to_contributions_dict)
    # the offset match
    offset_match = sum_sqrt_clusters_sums_squared - sum_contributions
    
    return offset_match

In [11]:
offset_match_output = offset_match(agents_to_contributions_dict, clusters_to_agents_dict_single_membership)
print(f"The offset match is {offset_match_output}")

The offset match is 5.19615242270663


# Multiple Memberships

## Cluster Match

In [12]:
# initialize a dictionary mapping each cluster id to the agents in that cluster
clusters_to_agents_dict = {}
# populate the dictionary
clusters_to_agents_dict[0] = ['A', 'S']
clusters_to_agents_dict[1] = ['A', 'B']
clusters_to_agents_dict[2] = ['S']

In [13]:
def get_agent_to_num_clusters_participating_dict(clusters_to_agents_dict):
    """
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: a dictionary mapping an agent to the number of clusters that that agent participates in
    """

    # initialize a dictionary mapping an agent to the number of clusters that that agent participates in
    agent_to_num_clusters_participating_dict = {}
    for cluster_key, cluster_agents in clusters_to_agents_dict.items():
        for cluster_agent in cluster_agents:
            # if an agent is in the current cluster, increment the number of clusters that they participate in by 1
            agent_to_num_clusters_participating_dict[cluster_agent] = agent_to_num_clusters_participating_dict.get(cluster_agent, 0) + 1
            
    return agent_to_num_clusters_participating_dict

In [14]:
def get_agent_to_contributions_to_cluster_dict(agents_to_contributions_dict, clusters_to_agents_dict, cluster_id):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param cluster_id: the id of the cluster from which we want each agent's contribution
    :return: a dictionary mapping an agent to their contribution to cluster cluster_id
    """

    # initialize a dictionary mapping an agent to their contribution to cluster cluster_id
    agent_to_contributions_to_cluster_dict = {}
    # get the agents in the cluster
    cluster_agents = clusters_to_agents_dict[cluster_id] 
    for cluster_agent in cluster_agents:
        # get the contribution of the current agent
        cluster_agent_contribution = agents_to_contributions_dict[cluster_agent]
        # add the agent and their contribution to the dictionary
        agent_to_contributions_to_cluster_dict[cluster_agent] = cluster_agent_contribution
    
    return agent_to_contributions_to_cluster_dict

In [15]:
def get_contribution_divided_by_num_affiliations(agents_to_contributions_dict, agent_to_num_clusters_participating_dict, agent):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param agent_to_num_clusters_participating_dict: a dictionary mapping an agent to the number of clusters that that agent participates in
    :param agent: the agent for which we want their contribution divided by the number of their affiliations
    :return: the agent's contribution divided by the number of their affiliations
    """
    
    # the agent's total contribution over all clusters
    agent_contribution = agents_to_contributions_dict[agent]
    # the number of affiliations the agent has
    num_agent_affiliations = agent_to_num_clusters_participating_dict[agent]
    # the agent's total contribution divided by the number of their affiliations
    agent_contribution_divided_by_num_affiliations = agent_contribution / num_agent_affiliations
    
    return agent_contribution_divided_by_num_affiliations

In [16]:
def cluster_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :return: cluster match with multiple affiliations
    """

    # dictionary mapping each agent to the number of clusters that that agent participates in
    agent_to_num_clusters_participating_dict = get_agent_to_num_clusters_participating_dict(clusters_to_agents_dict)
    # the sum of the elements inside the parentheses of the Multiple Membership Cluster Match
    sum_sqrt_clusters_sums = 0
    for cluster_key, cluster_agents in clusters_to_agents_dict.items():
        # initialize an empty list of the current cluster's agents' contributions divided by the agents' number of affiliations
        current_cluster_agent_contributions_divided_by_num_affiliations = []
        for cluster_agent in cluster_agents:
            # get the current cluster's agent's contribution divided by the number of that agent's affiliations
            current_cluster_agent_contribution_divided_by_num_affiliations = get_contribution_divided_by_num_affiliations(agents_to_contributions_dict, agent_to_num_clusters_participating_dict, cluster_agent)
            # append that value to the list
            current_cluster_agent_contributions_divided_by_num_affiliations.append(current_cluster_agent_contribution_divided_by_num_affiliations)
        # sum the current cluster's agents' contributions divided by the agents' number of affiliations
        sum_current_cluster_agent_contributions_divided_by_num_affiliations = np.sum(current_cluster_agent_contributions_divided_by_num_affiliations)
        # the square root of that number
        sqrt_sum_current_cluster_agent_contributions_divided_by_num_affiliations = np.sqrt(sum_current_cluster_agent_contributions_divided_by_num_affiliations)
        # add that value to the total sum inside the parentheses
        sum_sqrt_clusters_sums += sqrt_sum_current_cluster_agent_contributions_divided_by_num_affiliations
    # the sum in the parentheses squared
    sum_sqrt_clusters_sums_squared = np.power(sum_sqrt_clusters_sums, 2)
    # the sum of the contributions from the agents
    sum_contributions = get_sum_of_all_contributions_from_agents_to_contributions_dict(agents_to_contributions_dict)
    # cluster match with multiple affiliations
    cluster_match_with_multiple_affiliations = sum_sqrt_clusters_sums_squared - sum_contributions
    
    return cluster_match_with_multiple_affiliations

In [17]:
cluster_match_with_multiple_affiliations_output = cluster_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict)
print(f"The cluster match with multiple affiliations is {cluster_match_with_multiple_affiliations_output}")

The cluster match with multiple affiliations is 11.809220916344756


## Offset Match

In [18]:
def get_solution_to_system_of_linear_equations(A, b):
    """
    :param A: coefficient matrix
    :param b: column vector
    :return: solution to the system of linear equations defined by A and b
    """
        
    return np.linalg.solve(A, b)

In [19]:
coefficient_matrix_affiliations = np.array([
                       [1, 1/2, 1/4],
                       [1/2, 1, 0],
                       [1/4, 0, 1]
                      ])
b = np.array([1, 1, 1])

In [20]:
def offset_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict, coefficient_matrix_affiliations, b):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param coefficient_matrix_affiliations: coefficient matrix defining the affiliations
    :param b: column vector
    :return: offset match with multiple affiliations
    """

    # solution to the system of linear equations defined by the affiliation coefficient matrix and b
    solution_to_system_of_linear_equations = get_solution_to_system_of_linear_equations(coefficient_matrix_affiliations, b)
    # the term in parentheses squared
    term_on_the_left = np.power(np.sum(np.sqrt(np.multiply(solution_to_system_of_linear_equations, np.array(list(agents_to_contributions_dict.values()))))), 2)
    # the sum of the contributions from the agents
    sum_contributions = get_sum_of_all_contributions_from_agents_to_contributions_dict(agents_to_contributions_dict)
    # offset match with multiple affiliations
    offset_match_with_multiple_affiliations = term_on_the_left - sum_contributions
    
    return offset_match_with_multiple_affiliations

In [21]:
offset_match_with_multiple_affiliations_output = offset_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict, coefficient_matrix_affiliations, b)
print(f"The offset match with multiple affiliations is {offset_match_with_multiple_affiliations_output}")

The offset match with multiple affiliations is 6.486842291197528


# General Formulae

## Cluster Match

```cluster_match_with_multiple_affiliations``` already implements the general formula for cluster match.

## Offset Match

In [22]:
def get_num_common_affiliations(clusters_to_agents_dict, agent1, agent2):
    """
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agent1: the first agent
    :param agent2: the second agent
    :return: the number of affiliations that the first agent and the second agent have in common
    """
    
    # initialize the number of common affiliations
    num_common_affiliations = 0
    for cluster_key, cluster_agents in clusters_to_agents_dict.items():
        # if both agents are in the current cluster
        if (agent1 in cluster_agents) and (agent2 in cluster_agents):
            # increment the number of common affiliations
            num_common_affiliations += 1
            
    return num_common_affiliations

In [23]:
def get_correlation_score(clusters_to_agents_dict, agent1, agent2):
    """
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agent1: the first agent
    :param agent2: the second agent
    :return: the (ordered) correlation score between the first agent and the second agent
    """
    
    # dictionary mapping each agent to the number of clusters that that agent participates in
    agent_to_num_clusters_participating_dict = get_agent_to_num_clusters_participating_dict(clusters_to_agents_dict)
    # number of affiliations the first agent has
    num_clusters_agent1_participates_in = agent_to_num_clusters_participating_dict[agent1] 
    # the (ordered) correlation score between the first agent and the second agent
    correlation_score = get_num_common_affiliations(clusters_to_agents_dict, agent1, agent2) / num_clusters_agent1_participates_in
    
    return correlation_score

In [24]:
def get_coefficients_for_affiliation_matrix(agents_to_contributions_dict, clusters_to_agents_dict, agents):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agents: the agents 
    :return: the coefficients of the affiliation matrix
    """

    # initialize the coefficient matrix
    coefficient_matrix = []
    for agent_index, agent1 in enumerate(agents):
        # initialize the current row's coefficients
        current_row_coefficients = []
        for agent_index, agent2 in enumerate(agents):
            # get the (ordered) correlation score between the first agent and the second agent
            current_correlation_score = get_correlation_score(clusters_to_agents_dict, agent1, agent2)
            # add that correlation score to the current row's coefficients
            current_row_coefficients.append(current_correlation_score)
        # add that row of coefficients to the coefficient matrix
        coefficient_matrix.append(current_row_coefficients)
    
    return coefficient_matrix

In [25]:
def offset_match_general_formula(agents_to_contributions_dict, clusters_to_agents_dict, agents):
    """
    :param agents_to_contributions_dict: a dictionary mapping an agent id to their contribution
    :param clusters_to_agents_dict: a dictionary mapping a cluster id to the agents in that cluster
    :param agents: the agents 
    :return: the offset match derived from the general formula
    """

    # the coefficient matrix defining the affiliations
    coefficient_matrix = get_coefficients_for_affiliation_matrix(agents_to_contributions_dict, clusters_to_agents_dict, agents)
    # the column vector
    b = np.ones(len(agents))
    # the offset match
    offset_match_general_formula = offset_match_with_multiple_affiliations(agents_to_contributions_dict, clusters_to_agents_dict, coefficient_matrix_affiliations, b)
    
    return offset_match_general_formula

In [26]:
offset_match_general_formula_output = offset_match_general_formula(agents_to_contributions_dict, clusters_to_agents_dict, agents)
print(f"The offset match with multiple affiliations derived from the general formula is {offset_match_general_formula_output}")

The offset match with multiple affiliations derived from the general formula is 6.486842291197528


# Pairwise Matching

In [27]:
# matching cap
M = 10
# number of agents
num_agents = 5
# number of projects
num_projects = 2
# the minimum contribution 
min_contribution = 0
# the maximum contribution
max_contribution = 100
# a random matrix defining each agent's contribution to each project, bounded by the minimum and maximum contributions
contribution_matrix = np.random.randint(low=min_contribution, high=max_contribution, size=[num_agents, num_projects])

In [28]:
def get_correlation_elements(contribution_matrix):
    """
    :param contribution_matrix: a matrix defining each agent's contribution to each project
    :return: for each agent, and for each project, the square root of (their contribution * each other agent's contribution)
             i.e. the geometric mean of each pair of agents' contributions to each project
             with resulting shape (num agents, num agents, num projects)
    """

    # note: this calculation process can be made more efficient with numpy broadcasting
    # initialize the resulting matrix
    correlation_elements = []
    for row1_index, row1 in enumerate(contribution_matrix):
        # initialize the current row's elements
        row1_correlation_elements = []
        for row2_index, row2 in enumerate(contribution_matrix):
            # the square roots of the pointwise multiplication between the contributions 
            # in the row in the outer loop and the contributions in the row in the inner loop
            current_correlation_elements = np.sqrt(np.multiply(row1, row2))
            # add the current correlation elements from the current outer and inner loop rows to the outer loop row's correlation elements
            row1_correlation_elements.append(current_correlation_elements)
        # add the correlation elements for the current row to resulting matrix
        correlation_elements.append(row1_correlation_elements)
        
    return correlation_elements

In [29]:
def get_correlation_scores(contribution_matrix, correlation_elements):
    """
    :param contribution_matrix: a matrix defining each agent's contribution to each project
    :param correlation_elements: for each agent, and for each project, the square root of (their contribution * each other agent's contribution)
                                 i.e. the geometric mean of each pair of agents' contributions to each project, with shape (num agents, num agents, num projects)
    :return: the correlation scores between all the agents, with shape (num agents, num agents)
    """

    # initialize the resulting matrix
    correlation_scores = []
    for row1_index, row1 in enumerate(contribution_matrix):
        # initialize the list of correlation scores between the agent in the outer loop and all the other agents
        row1_correlation_scores = []
        for row2_index, row2 in enumerate(contribution_matrix):
            # calculate the correlation score between the agent in the outer loop and the agent in the inner loop
            correlation_score = np.sum(correlation_elements[row1_index][row2_index])
            # add the current correlation score to the agent in the outer loop's list of correlation scores 
            row1_correlation_scores.append(correlation_score)
        # add the agent in the outer loop's correlation scores to the resulting matrix
        correlation_scores.append(row1_correlation_scores)

    return correlation_scores

In [30]:
def pairwise_matching(contribution_matrix, num_agents, num_projects):
    """
    :param contribution_matrix: a matrix defining each agent's contribution to each project
    :param num_agents: the number of agents
    :param num_projects: the number of projects                       
    :return: the match matrix, with shape (num agents, num agents, num projects)
    """

    # the resulting matrix
    pairwise_matching_matrix = np.zeros(shape=[num_agents, num_agents, num_projects])
    # the correlation elements
    correlation_elements = get_correlation_elements(contribution_matrix)
    # the correlation scores
    correlation_scores = get_correlation_scores(contribution_matrix, correlation_elements)
    for agent1_index in range(num_agents):
        for agent2_index in range(num_agents):
            for project_index in range(num_projects):
                # calculate the match matrix entry for the match for the outer loop agent, the inner loop agent, and the current project
                # note: since we are summing over ordered pairs, we use M as the coefficient, rather than 2M
                pairwise_matching_matrix[agent1_index, agent2_index, project_index] = (M * correlation_elements[agent1_index][agent2_index][project_index]) / (M + correlation_scores[agent1_index][agent2_index])
                
    return pairwise_matching_matrix

In [31]:
pairwise_matching_matrix_output = pairwise_matching(contribution_matrix, num_agents, num_projects)
print(f"The pairwise matching is \n\n {pairwise_matching_matrix_output}")

The pairwise matching is 

 [[[5.62962963 3.62962963]
  [5.09613983 3.99091938]
  [5.91459781 3.39296114]
  [5.74347306 3.48009831]
  [4.71414767 4.50534719]]

 [[5.09613983 3.99091938]
  [4.55555556 4.33333333]
  [5.38739228 3.75388875]
  [5.20824486 3.83316443]
  [4.19065406 4.86469956]]

 [[5.91459781 3.39296114]
  [5.38739228 3.75388875]
  [6.19354839 3.16129032]
  [6.02625782 3.24889602]
  [5.00585419 4.25671385]]

 [[5.74347306 3.48009831]
  [5.20824486 3.83316443]
  [6.02625782 3.24889602]
  [5.85365854 3.33333333]
  [4.83494591 4.34261309]]

 [[4.71414767 4.50534719]
  [4.19065406 4.86469956]
  [5.00585419 4.25671385]
  [4.83494591 4.34261309]
  [3.80952381 5.3968254 ]]]
